In [1]:
import os
import numpy as np
import time
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.simplefilter("ignore")

In [2]:
MAX_ITER = 10
BATCH_SIZE = 30
LEARNING_RATE = 1e-3
TRAIN_DATA_PATH = "train/"
VAL_DATA_PATH = "val/"
NUM_CLASSES = 2

TRANSFORM = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(1, 1.5), saturation=(1, 1.2)),
    transforms.ToTensor(),
    ])


train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
val_data = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, transform=TRANSFORM)
val_data_loader = data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataloaders = {'train': train_data_loader, 'val': val_data_loader}
dataset_sizes = {'train': len(train_data),'val':len(val_data)}

In [3]:
print(dataset_sizes)

{'train': 26430, 'val': 2981}


In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            epoch_pred = []
            epoch_truth = []

            batch = 0
            for inputs, labels in dataloaders[phase]:
                batch += 1
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                epoch_truth.extend(labels.cpu().data)
                epoch_pred.extend(preds.cpu())
                
                if batch % 50 == 0 and phase == 'train':
                    print('Batch {} Loss {:.4f}'.format(batch, loss.item()))
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_f1 = f1_score(epoch_truth, epoch_pred, average='weighted')
            epoch_auc = roc_auc_score(epoch_truth, epoch_pred)
                          
            print('{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f} AUC {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1,
                                                                            epoch_auc))
            
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())

        print('-'*5)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Val F1: {:4f}'.format(best_f1))

    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = models.densenet161(pretrained=True)

print(model_ft.classifier)

Linear(in_features=2208, out_features=1000, bias=True)


In [6]:
model_ft.classifier = nn.Linear(2208, NUM_CLASSES)

model_ft = model_ft.to(device)

w = [0.10598645840316419, 1.894013541596836]

weights = torch.tensor(w).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)

optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.5)

In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=MAX_ITER)

Epoch 0/9
----------
Batch 50 Loss 0.9040
Batch 100 Loss 0.8387
Batch 150 Loss 0.7042
Batch 200 Loss 0.5293
Batch 250 Loss 0.6165
Batch 300 Loss 0.5541
Batch 350 Loss 0.6498
Batch 400 Loss 0.7129
Batch 450 Loss 0.7642
Batch 500 Loss 0.6122
Batch 550 Loss 0.7195
Batch 600 Loss 0.6340
Batch 650 Loss 0.7975
Batch 700 Loss 0.6254
Batch 750 Loss 0.6086
Batch 800 Loss 0.7044
Batch 850 Loss 0.7005
train Loss: 0.7187 Acc: 0.7221 F1: 0.7967 AUC 0.5403
val Loss: 0.9604 Acc: 0.9507 F1: 0.9267 AUC 0.5000
-----
Epoch 1/9
----------
Batch 50 Loss 0.6352
Batch 100 Loss 0.6392
Batch 150 Loss 0.6361
Batch 200 Loss 0.7957
Batch 250 Loss 1.1801
Batch 300 Loss 0.5365
Batch 350 Loss 0.6545
Batch 400 Loss 0.5647
Batch 450 Loss 0.7471
Batch 500 Loss 0.6229
Batch 550 Loss 0.7538
Batch 600 Loss 0.7950
Batch 650 Loss 0.7950
Batch 700 Loss 0.6414
Batch 750 Loss 0.6833
Batch 800 Loss 0.7025
Batch 850 Loss 0.6403
train Loss: 0.6878 Acc: 0.7702 F1: 0.8276 AUC 0.5315
val Loss: 0.6798 Acc: 0.9403 F1: 0.9225 AUC 0.501

In [8]:
TEST_DATA_PATH = "test/"
WA_DATA_PATH = "test_wa/"
DEYE_DATA_PATH = "test_deye/"

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

wa_data = torchvision.datasets.ImageFolder(root=WA_DATA_PATH, transform=TRANSFORM)
wa_data_loader = data.DataLoader(wa_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

deye_data = torchvision.datasets.ImageFolder(root=DEYE_DATA_PATH, transform=TRANSFORM)
deye_data_loader = data.DataLoader(deye_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)


dataloaders = {'test_kaggle': test_data_loader, 'test_wa': wa_data_loader, 'test_deye': deye_data_loader}
dataset_sizes = {'test_kaggle': len(test_data),'test_wa':len(wa_data), 'test_deye':len(deye_data)}

In [9]:
print(dataset_sizes)

{'test_kaggle': 44837, 'test_wa': 45, 'test_deye': 38}


In [10]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt


def my_plot(truth, pred):
    auc = roc_auc_score(truth, pred)
    print('AUC: %.4f' % auc)
    return

In [15]:
def eval_model(model, criterion, optimizer, scheduler):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_f1 = 0.0

    for epoch in range(1):

        # Each epoch has a training and validation phase
        # for phase in ['test_kaggle', 'test_wa', 'test_deye']:
        for phase in ['test_wa', 'test_deye']:
            model.eval()   # Set model to evaluate mode
            print('Evaluating {}'.format(phase))
            running_loss = 0.0
            running_corrects = 0
            epoch_pred = []
            epoch_truth = []

            # Iterate over data.
            batch = 0
            # batch_start = time.time()
            for inputs, labels in dataloaders[phase]:
                batch += 1
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                epoch_truth.extend(labels.cpu().data)
                epoch_pred.extend(preds.cpu())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_f1 = f1_score(epoch_truth, epoch_pred, average='weighted')
                          
            print('{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1))
            print(classification_report(epoch_truth, epoch_pred))
            print(confusion_matrix(epoch_truth, epoch_pred))
            # my_plot(epoch_truth, epoch_pred)
            
        print('-'*5)
    return

In [16]:
eval_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

Evaluating test_wa
test_wa Loss: 0.7983 Acc: 0.0000 F1: 0.0000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00        45

   micro avg       0.00      0.00      0.00        45
   macro avg       0.00      0.00      0.00        45
weighted avg       0.00      0.00      0.00        45

[[ 0  0]
 [45  0]]
Evaluating test_deye
test_deye Loss: 0.8158 Acc: 0.4737 F1: 0.3045
              precision    recall  f1-score   support

           0       0.47      1.00      0.64        18
           1       0.00      0.00      0.00        20

   micro avg       0.47      0.47      0.47        38
   macro avg       0.24      0.50      0.32        38
weighted avg       0.22      0.47      0.30        38

[[18  0]
 [20  0]]
-----


In [ ]:
torch.save(model_ft.state_dict(), 'nova_dense.pth')